In [46]:
import tensorflow as tf
import keras
import numpy as np
from tqdm import tqdm
from keras.layers import LSTM, Conv1D, Flatten, Dropout, Dense, InputLayer, ZeroPadding2D, Convolution2D, MaxPooling2D, Conv2D
from keras.layers.embeddings import Embedding
from keras.models import Sequential
from keras.regularizers import l2 as L2
import pandas as pd
import os
import cv2
import matplotlib.pyplot as plt
import pickle

l2 = L2(0.01)

## Images analysis

In [121]:
# Batch generator
import numpy as np

class DataGenerator(object):
    
    def __init__(self, batch_size = 32, dim_x = 224, dim_y = 224, dim_z = 3, shuffle = True):
        self.dim_x = dim_x
        self.dim_y = dim_y
        self.dim_z = dim_z
        self.batch_size = batch_size
        self.shuffle = shuffle

    def generate(self, labels, pic_names):
        # Infinite loop
        while 1:
        # Generate order of exploration of dataset
            indexes = self.__get_exploration_order(pic_names)

            # Generate batches
            imax = int(len(indexes)/self.batch_size)
            for i in range(imax):
                # Find list of IDs
                list_IDs_temp = [pic_names[k] for k in indexes[i*self.batch_size:(i+1)*self.batch_size]]

                # Generate data
                X, y = self.__data_generation(labels, list_IDs_temp)

                yield X, y

    def __get_exploration_order(self, pic_names):
        
        # Find exploration order
        indexes = np.arange(len(pic_names))
        if self.shuffle == True:
            np.random.shuffle(indexes)

        return indexes

    def __data_generation(self, labels, list_IDs_temp):
        # X : (n_samples, v_size, v_size, v_size, n_channels)
        # Initialization
        X = np.empty((self.batch_size, self.dim_x, self.dim_y, self.dim_z))
        y = np.empty((self.batch_size), dtype = np.float32)

        # Generate data
        for i, ID in enumerate(list_IDs_temp):
            # Store volume
            img = cv2.imread('Dataset/images/'+ID)
            img = cv2.resize(img,(224,224))
            img = img/255
            #print(img)
            X[i, :, :, :] = img
            #print(X)

            # Store class
            y[i] = labels[labels.id == ID].likes#labels[int(ID[:-4])]
            #print(labels.iloc[int(ID[:-4])].likes)
        return X, y
    def debug(self, labels, pic_names):
        indexes = self.__get_exploration_order(pic_names)
        imax = int(len(indexes)/self.batch_size)
        
        for i in range(1):
            # Find list of IDs
            list_IDs_temp = [pic_names[k] for k in indexes[i*self.batch_size:(i+1)*self.batch_size]]

            # Generate data
            X, y = self.__data_generation(labels, list_IDs_temp)
            print(X.shape)
DataGenerator().debug(dataset_train, train_names)

(32, 224, 224, 3)


In [120]:
dataset_train[dataset_train.id == '3453.jpg'].likes.values

array([ 0.05645522])

In [110]:
#Loading data
data = pd.read_csv('Dataset/dataset_raw.csv')
data.likes = data.likes/data.likes.max()

with open ('Dataset/dataset_train.pkl', 'rb') as f:
    dataset_train = pickle.load(f)
with open ('Dataset/dataset_val.pkl', 'rb') as f:
    dataset_val = pickle.load(f)

train_names = dataset_train.id.values
val_names = dataset_val.id.values

In [24]:
# pic_names = os.listdir('Dataset/images/')
# msk = np.random.rand(len(pic_names)) < 0.8
# train_names = np.array(pic_names)[msk]
# val_names = np.array(pic_names)[~msk]

# msk = np.random.rand(len(val_names)) < 0.5
# test_names = val_names[msk]
# val_names = val_names[~msk]

# print(train_names.shape)
# print(val_names.shape)
# print(test_names.shape)

(12539,)
(1547,)
(1628,)


In [5]:
# train_y = np.array([data.iloc[int(i[:-4])].likes for i in train_names])
# val_y = np.array([data.iloc[int(i[:-4])].likes for i in val_names])
# print(train_y.shape)
# print(val_y.shape)

(12450,)
(3264,)


In [61]:
# # load images
# train_X =[]
# val_X=[]
# for pic in train_names:
#     img = cv2.imread('Dataset/images/'+pic)
#     img = cv2.resize(img, (224,224))
#     img = img/255
#     train_X.append(img)
# for pic in val_names:
#     img = cv2.imread('Dataset/images/'+pic)
#     img = cv2.resize(img, (224,224))
#     img = img/255
#     val_X.append(img)
# train_X = np.array()

(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 

(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 

KeyboardInterrupt: 

In [122]:
img_input = keras.layers.Input(shape=(224,224,3))
x = Conv2D(64, (3, 3), activation='relu', padding='same', name='block1_conv1')(img_input)
x = Conv2D(64, (3, 3), activation='relu', padding='same', name='block1_conv2')(x)
x = MaxPooling2D((2, 2), strides=(2, 2), name='block1_pool')(x)

# Block 2
x = Conv2D(128, (3, 3), activation='relu', padding='same', name='block2_conv1')(x)
x = Conv2D(128, (3, 3), activation='relu', padding='same', name='block2_conv2')(x)
x = MaxPooling2D((2, 2), strides=(2, 2), name='block2_pool')(x)

# Block 3
x = Conv2D(256, (3, 3), activation='relu', padding='same', name='block3_conv1')(x)
x = Conv2D(256, (3, 3), activation='relu', padding='same', name='block3_conv2')(x)
x = Conv2D(256, (3, 3), activation='relu', padding='same', name='block3_conv3')(x)
x = MaxPooling2D((2, 2), strides=(2, 2), name='block3_pool')(x)

# Block 4
x = Conv2D(512, (3, 3), activation='relu', padding='same', name='block4_conv1')(x)
x = Conv2D(512, (3, 3), activation='relu', padding='same', name='block4_conv2')(x)
x = Conv2D(512, (3, 3), activation='relu', padding='same', name='block4_conv3')(x)
x = MaxPooling2D((2, 2), strides=(2, 2), name='block4_pool')(x)

# Block 5
x = Conv2D(512, (3, 3), activation='relu', padding='same', name='block5_conv1')(x)
x = Conv2D(512, (3, 3), activation='relu', padding='same', name='block5_conv2')(x)
x = Conv2D(512, (3, 3), activation='relu', padding='same', name='block5_conv3')(x)
x = MaxPooling2D((2, 2), strides=(2, 2), name='block5_pool')(x)

x = Flatten(name='flatten')(x)
x = Dense(4096, activation='relu', name='fc1')(x)
x = Dense(2048, activation='relu', name='fc2')(x)
output = Dense(1, activation=None, name='predictions')(x)
model = keras.models.Model(img_input, output, name='model_1')
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_8 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0         
__________

In [127]:
optimizer = keras.optimizers.SGD(lr=0.00001)
model.compile(loss='mse', optimizer=optimizer, metrics=['mse'])
batch_size = 16
training_generator = DataGenerator(batch_size = batch_size).generate(dataset_train, train_names)
validation_generator = DataGenerator(batch_size = batch_size).generate(dataset_val, val_names)

In [128]:
model.fit_generator(generator = training_generator,
                    steps_per_epoch = len(train_names)//batch_size,
                    validation_data = validation_generator,
                    validation_steps = len(val_names)//batch_size, verbose = True,epochs= 10)
#model.fit(X_train_emb, y_train,validation_data=(X_val_emb, y_val),verbose=True, epochs=20)

Epoch 1/10
502/779 [==================>...........] - ETA: 251s - loss: 0.0041 - mean_squared_error: 0.0041

KeyboardInterrupt: 

In [129]:
model.save('saved_models/image_model_v1.h5')

In [59]:
img = cv2.imread('Dataset/images/2858.jpg')
img = cv2.resize(img, (224,224))
img = img/255

img =np.expand_dims(img,axis = 0)
print(model.predict(img))
print(data.iloc[2858].likes)

[[ 0.05503913]]
0.117081010701


## Texts analysis

In [130]:
with open ('Dataset/dataset_train.pkl', 'rb') as f:
    dataset_train = pickle.load(f)
with open ('Dataset/dataset_val.pkl', 'rb') as f:
    dataset_val = pickle.load(f)

In [131]:
X_train = dataset_train.texts.values
y_train = dataset_train.likes.values

X_val = dataset_val.texts.values
y_val = dataset_val.likes.values
y_val

array([ 0.28431053,  0.04317044,  0.01828946, ...,  0.18059568,
        0.15221549,  0.02284656])

In [132]:
def char_to_vocab (sentence):
    vocabulary = ['а','б','в','г','д','е','ё','ж','з','и','й','к','л','м','н','о','п','р','с','т','у','ф','х','ц','ч','ш','щ','ъ','ы','ь','э','ю','я',
              'А','Б','В','Г','Д','Е','Ё','Ж','З','И','Й','К','Л','М','Н','О','П','Р','С','Т','У','Ф','Х','Ц','Ч','Ш','Щ','Ъ','Ы','Ь','Э','Ю','Я',
             '!','@','#','$','%','^','&','*','(',')',':',';','/',',','.','%','№','?','~','-','+','=',' ',
              'a','b','c','d','e','f','g','h','i','j','k','l','m','n','o','p','q','r','s','t','u','v','w','x','y','z',
             'A','B','C','D','E','F','G','H','I','J','K','L','M','N','O','P','Q','R','S','T','U','V','W','X','Y','Z']
    result = []
    for char in sentence:
        if char in vocabulary:
            result.append(vocabulary.index(char)+1)
            
    result = np.array(result)
    #encoded = one_hot(sentence)
    num_of_lett = 300
    if len(result)<num_of_lett:
        result = np.concatenate((result, np.zeros(num_of_lett-result.shape[0])))
    if len(result)>num_of_lett:
        result = result[:num_of_lett]
    return list(result)

X_train_encoded = []
for sentence in tqdm(X_train):
    X_train_encoded.append(char_to_vocab(sentence))
X_train = np.array(X_train_encoded)

X_val_encoded = []
for sentence in tqdm(X_val):
    X_val_encoded.append(char_to_vocab(sentence))
X_val = np.array(X_val_encoded)

100%|██████████| 1497/1497 [00:00<00:00, 3576.87it/s]


In [133]:
X_train.shape

(12473, 300)

In [135]:
input = keras.layers.Input(shape=(X_train.shape[1],))
l_init = keras.layers.embeddings.Embedding(300, 80, input_length=300)(input)
l_0_0 = keras.layers.Dense(128)(l_init) 
l_0_1 = keras.layers.Conv1D(filters=128, kernel_size=2, activation='relu', kernel_regularizer=l2, padding='same')(l_init) 
l_0_2 = keras.layers.Conv1D(filters=256, kernel_size=4, activation='relu', kernel_regularizer=l2,padding='same')(l_init) 
l_0_3 = keras.layers.Conv1D(filters=256, kernel_size=8, activation='relu', kernel_regularizer=l2,padding='same')(l_init)
l_0_4 = keras.layers.Conv1D(filters=512, kernel_size=16, activation='relu', kernel_regularizer=l2,padding='same')(l_init) 
l_0_5 = keras.layers.Conv1D(filters=512, kernel_size=32, activation='relu', kernel_regularizer=l2,padding='same')(l_init) 

merge = keras.layers.Merge(mode='concat', concat_axis=2) ([l_0_0, l_0_1, l_0_2, l_0_3,l_0_4,l_0_5]) 
l_5 = keras.layers.GlobalMaxPooling1D()(merge) 
l_6 = keras.layers.Dropout(0.5)(l_5) 
output = keras.layers.Dense(1, activation=None, kernel_regularizer=l2)(l_6) 
model = keras.models.Model(input, output, name='model_5')
print(model.summary())

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_10 (InputLayer)            (None, 300)           0                                            
____________________________________________________________________________________________________
embedding_5 (Embedding)          (None, 300, 80)       24000       input_10[0][0]                   
____________________________________________________________________________________________________
dense_9 (Dense)                  (None, 300, 128)      10368       embedding_5[0][0]                
____________________________________________________________________________________________________
conv1d_21 (Conv1D)               (None, 300, 128)      20608       embedding_5[0][0]                
___________________________________________________________________________________________

/home/ludapak/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:10: UserWarning: The `Merge` layer is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.
  # Remove the CWD from sys.path while we load stuff.


In [138]:
optimizer = keras.optimizers.SGD(lr=0.0001)
model.compile(loss='mse', optimizer=optimizer, metrics=['mse'])
model.fit(X_train, y_train,validation_data=(X_val, y_val),verbose=True, epochs=20)

Train on 12473 samples, validate on 1497 samples
Epoch 1/20
12473/12473 [==============================] - 29s - loss: 5.7203 - mean_squared_error: 0.0071 - val_loss: 5.7128 - val_mean_squared_error: 0.0040
Epoch 2/20
12473/12473 [==============================] - 28s - loss: 5.7116 - mean_squared_error: 0.0072 - val_loss: 5.7039 - val_mean_squared_error: 0.0040
Epoch 3/20
12473/12473 [==============================] - 28s - loss: 5.7024 - mean_squared_error: 0.0070 - val_loss: 5.6950 - val_mean_squared_error: 0.0040
Epoch 4/20
12473/12473 [==============================] - 28s - loss: 5.6936 - mean_squared_error: 0.0070 - val_loss: 5.6861 - val_mean_squared_error: 0.0040
Epoch 5/20
12473/12473 [==============================] - 28s - loss: 5.6848 - mean_squared_error: 0.0071 - val_loss: 5.6772 - val_mean_squared_error: 0.0040
Epoch 6/20
 3712/12473 [=======>......................] - ETA: 19s - loss: 5.6789 - mean_squared_error: 0.0070

KeyboardInterrupt: 

In [139]:
model.save('saved_models/text_model_v1.h5')

In [ ]:
with open ('Dataset/dataset_test.pkl', 'rb') as f:
    dataset_test = pickle.load(f)
    
X_test = dataset_test.texts.values
y_test = dataset_test.likes.values

X_test_encoded = []
for sentence in tqdm(X_test):
    X_test_encoded.append(char_to_vocab(sentence))
X_test = np.array(X_test_encoded)



print(model.predict(X_test[100:105]))
print(y_test[100:105])